<a href="https://colab.research.google.com/github/muhamadrefo/ImageClassificationColab/blob/master/CSV_v2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#Importing Packages
seedNum = 1
import numpy as np
import matplotlib.pyplot as plt
import glob
import math
import cv2
import os
import shutil
import sys
import pandas as pd
import random
random.seed(seedNum)
import tensorflow as tf
tf.random.set_seed(seedNum)

from PIL import Image
from PIL import ImageDraw
from datetime import datetime
from skimage.transform import resize
from skimage.feature import greycomatrix, greycoprops
from matplotlib.image import imread
from datetime import datetime
%matplotlib inline

In [10]:
%cd /content/drive/My Drive/Dataset/Batik/

/content/drive/My Drive/Dataset/Batik


In [5]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
# create directories
dataset_home = 'YogyaNogosari/'
subdirs = ['train/', 'valid/']
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['cap/', 'tulis/']
	for labldir in labeldirs:
		newdir = dataset_home + subdir + labldir
		makedirs(newdir, exist_ok=True)
# seed random number generator
seed(seedNum)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = '/content/drive/My Drive/Batik/CNN/dataset/Training_nogosari/'
for file in listdir(src_directory):
	src = src_directory + '/' + file
	dst_dir = 'train/'
	if random() < val_ratio:
		dst_dir = 'valid/'
	if file.startswith('Yogya-Tulis'):
		dst = dataset_home + dst_dir + 'tulis/'  + file
		copyfile(src, dst)
	elif file.startswith('Yogya-Cap'):
		dst = dataset_home + dst_dir + 'cap/'  + file
		copyfile(src, dst)

In [12]:
map_8bit_to_3bit = [i // 32 for i in range(256)]

def read_file(myDir):
  file_name = []
  label = []
  namePath = []

  for path, subdirs, files in os.walk(myDir):
    for name in files:
      namePath.append(os.path.join(path, name))
      label.append(path.split('/')[-1])

  return namePath, label


def load_img(img_path):
  img = Image.open(img_path).convert('L')
  resultImg = img.resize((224, 224))
  return resultImg


def show_img(img):
  plt.imshow(img, cmap='gray')

def getLabels(path):
  label = os.listdir(path)
  return label
  
#Preprocessing start from here

def getColoringIMG(img):
  ColoringIMG = list(img.getdata())
  return ColoringIMG


def mapTo3bit(ColoringIMG):
  i = 0

  for colorIMG in ColoringIMG:
    ColoringIMG[i] = map_8bit_to_3bit[colorIMG]
    i += 1

  return ColoringIMG


def transToMatriks(ColoringIMG):
  matriksIMG = []
  i = 0

  dimensionIMG = int(math.sqrt(len(ColoringIMG)))

  for row in range(dimensionIMG):
    newRow = []
    for col in range(dimensionIMG):
      newRow.append(ColoringIMG[i])
      i += 1
    matriksIMG.append(newRow)

  return matriksIMG

def ExtractIMG(img):
  IMGColor = getColoringIMG(img)
  _3BitColor = mapTo3bit(IMGColor)
  matriksIMG = transToMatriks(_3BitColor)

  GLCMresult = []
  glcm = greycomatrix(matriksIMG, distances=[1],angles=[0],levels=256,symmetric=False,normed=False)
  komponen = ['dissimilarity', 'contrast', 'homogeneity', 'energy', 'correlation', 'ASM']

  for i in komponen:
    GLCMresult.append(greycoprops(glcm, i)[0][0])

  return GLCMresult

In [17]:
def makeTrain():
  Root = '/content/drive/My Drive/Dataset/Batik/YogyaNogosari/train/'
  startTimeModule = datetime.now()

  path_trainIMG = []
  label_trainIMG = []
  dissimilarity = []
  contrast = []
  homogeneity = []
  energy = []
  correlation = []
  ASM = []

  labels = getLabels(Root)

  for label in labels:
    names_Path = glob.glob(Root + label + '/' + '*.jpg')
    for namePath in names_Path:
      path_trainIMG.append(namePath)
      label_trainIMG.append(label)
      trainImg = load_img(namePath)
      featureOfImg = ExtractIMG(trainImg)
      dissimilarity.append(featureOfImg[0])
      contrast.append(featureOfImg[1])
      homogeneity.append(featureOfImg[2])
      energy.append(featureOfImg[3])
      correlation.append(featureOfImg[4])
      ASM.append(featureOfImg[5])

  columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
  #['Label','Path', 'Homogeneity', 'Dissimilarity', 'Contrast', 
  #           'Energy', 'Correlation', 'ASM']
  variables = {
      'Path':path_trainIMG,
      'Contrast':contrast,
      'Dissimilarity':dissimilarity,
      'Homogeneity':homogeneity,
      'Energy':energy,
      'Correlation':correlation,
      'ASM':ASM,
      'Label':label_trainIMG,
              }

  data_train = pd.DataFrame(variables, columns = columns)
  #data_train.to_csv("DataTraining.csv", index=False) #Changes DataTraining.csv to DataValidation.csv or vice versa
  data_train.to_csv("DataTraining.csv", index=False)
  print('Total time for processing :', (datetime.now() - startTimeModule))

In [23]:
def makeValid():
  Root = '/content/drive/My Drive/Dataset/Batik/YogyaNogosari/valid/'
  startTimeModule = datetime.now()

  path_trainIMG = []
  label_trainIMG = []
  dissimilarity = []
  contrast = []
  homogeneity = []
  energy = []
  correlation = []
  ASM = []

  labels = getLabels(Root)

  for label in labels:
    names_Path = glob.glob(Root + label + '/' + '*.jpg')
    for namePath in names_Path:
      path_trainIMG.append(namePath)
      label_trainIMG.append(label)
      trainImg = load_img(namePath)
      featureOfImg = ExtractIMG(trainImg)
      dissimilarity.append(featureOfImg[0])
      contrast.append(featureOfImg[1])
      homogeneity.append(featureOfImg[2])
      energy.append(featureOfImg[3])
      correlation.append(featureOfImg[4])
      ASM.append(featureOfImg[5])

  columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
  #['Label','Path', 'Homogeneity', 'Dissimilarity', 'Contrast', 
  #           'Energy', 'Correlation', 'ASM']
  variables = {
      'Path':path_trainIMG,
      'Contrast':contrast,
      'Dissimilarity':dissimilarity,
      'Homogeneity':homogeneity,
      'Energy':energy,
      'Correlation':correlation,
      'ASM':ASM,
      'Label':label_trainIMG,
              }

  data_train = pd.DataFrame(variables, columns = columns)
  #data_train.to_csv("DataTraining.csv", index=False) #Changes DataTraining.csv to DataValidation.csv or vice versa
  data_train.to_csv("DataValidation.csv", index=False)
  print('Total time for processing :', (datetime.now() - startTimeModule))

In [24]:
makeTrain()

Total time for processing : 0:00:13.931867


In [25]:
show = pd.read_csv('DataTraining.csv')
show

,Path,Dissimilarity,Contrast,Homogeneity,Energy,Correlation,ASM,Label
0,/content/drive/My Drive/Dataset/Batik/YogyaNog...,1.161895,2.390715,0.539011,0.194529,0.605526,0.037841,cap
1,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.594090,0.842489,0.727348,0.379189,0.437346,0.143784,cap
2,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.515515,0.679753,0.758610,0.382817,0.603333,0.146549,cap
3,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.548847,0.727018,0.743285,0.385256,0.536179,0.148422,cap
4,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.552010,0.777266,0.746220,0.384418,0.539110,0.147778,cap
...,...,...,...,...,...,...,...,...
407,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.585122,0.878163,0.736474,0.361898,0.609560,0.130970,tulis
408,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.418081,0.518938,0.801011,0.419590,0.644541,0.176056,tulis
409,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.494775,0.771320,0.780177,0.344513,0.849075,0.118689,tulis
410,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.499840,0.738189,0.773836,0.511502,0.325221,0.261634,tulis


In [26]:
makeValid()

Total time for processing : 0:00:04.481989


In [27]:
valid = pd.read_csv('DataValidation.csv')
valid

,Path,Dissimilarity,Contrast,Homogeneity,Energy,Correlation,ASM,Label
0,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.179432,0.223555,0.914651,0.698217,0.793293,0.487508,cap
1,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.202014,0.240171,0.902809,0.622408,0.838682,0.387391,cap
2,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.467369,0.628563,0.782357,0.494227,0.613842,0.244261,cap
3,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.589666,0.925589,0.738607,0.272612,0.788963,0.074317,cap
4,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.627122,0.905349,0.714101,0.363739,0.449124,0.132306,cap
...,...,...,...,...,...,...,...,...
128,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.243894,0.257067,0.879370,0.499696,0.786246,0.249696,tulis
129,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.491972,0.789017,0.783602,0.355722,0.842123,0.126538,tulis
130,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.318526,0.355281,0.844413,0.420870,0.805630,0.177131,tulis
131,/content/drive/My Drive/Dataset/Batik/YogyaNog...,0.456398,0.736147,0.799538,0.395098,0.852358,0.156102,tulis
